# Importación y lectura de las bases de datos

### Con big-query

In [ ]:
from google.cloud import bigquery                      # librerias para trabajar con bigquery
from google.oauth2 import service_account

# direccion local de archivo JSON con las credenciales del proyecto
path_pk = 'python_pk_trabajo_analitica.json'           
         
credentials = service_account.Credentials.from_service_account_file(path_pk)         # lectura credenciales
project_id = 'sunlit-inn-353723'                                                     # id del projecto
client = bigquery.Client(credentials= credentials, project=project_id)               # creacion del client para trabajar en python

In [ ]:
# querys para las tablas a importar desde bigquery
query_cv = "SELECT * FROM `sunlit-inn-353723.trabajo_final_analitica.condiciones_vida`"     
query_vi = "SELECT * FROM `sunlit-inn-353723.trabajo_final_analitica.violen_intfam`"
query_sc = "SELECT * FROM `sunlit-inn-353723.trabajo_final_analitica.suicidios`"

# importacion de las tablas como df's
cv_df = client.query(query_cv).to_dataframe()             # df para condiciones de vida
vi_df = client.query(query_vi).to_dataframe()             # df para violencia intrafamiliar
sc_df = client.query(query_sc).to_dataframe()             # df para suicidios

### Desde la dirección de los datos

In [1]:
import pandas as pd
import unidecode as udc
pd.options.display.float_format = '{:.2f}'.format

# lectura de las bases de datos como data-frame con la dirección directa de las bases de datos
cv_df = pd.read_csv('https://datosabiertos.bogota.gov.co/dataset/5e79c701-d02f-4220-a9ae-31c8def1cfab/resource/47c5207a-d597-4c9e-ad74-4fb980656bf8/download/osb_demografia-condicionesvida.csv', encoding='latin-1', delimiter=';', decimal=',')
vi_df = pd.read_csv('https://datosabiertos.bogota.gov.co/dataset/a1e1ef90-10c0-436f-a290-d1f7c1cf2242/resource/ab4eeb6e-e7c1-4ec1-b3e5-eedc655bc8d7/download/osb_v-intrafamiliar.csv', encoding='latin-1', delimiter=';', decimal=',', thousands='.')
sc_df = pd.read_csv('https://datosabiertos.bogota.gov.co/dataset/2b8464e3-3aca-4dcd-91a1-93dd06ddabbb/resource/f215cedd-46e0-44fe-ba4c-704afdc11a33/download/osb_saludmen-tsuicidiodesagregado.csv', encoding='latin-1', delimiter=';')


# 'CLeaning' en los nombres y tipos de cada columna para un mejor acceso
vi_df['Año'] = vi_df['Año'].str.extract(r'(\d{4})').astype(int)
vi_df.columns = [udc.unidecode(i) for i in vi_df.columns.str.lower().str.replace('.', '', regex=False).str.replace(' ', '_', regex=False)]
cv_df.columns = cv_df.columns.str.lower().str.replace(' ', '_')
sc_df.columns = [udc.unidecode(i) for i in sc_df.columns.str.lower().str.replace(' ', '_', regex=False)]


dfs = [cv_df, vi_df, sc_df]              # lista de data-frames
for i in dfs:
    if 'ano' in i.columns:
        i.rename(columns={'ano': 'year'}, inplace=True)
        

# Aplicacion de funciones

## Condiciones de vida

Descripción de la variable porcentaje por cada tipo de _**condiciones_de_vida**_

In [2]:
cv_df.groupby('condiciones_de_vida')['porcentaje'].describe()

,count,mean,std,min,25%,50%,75%,max
condiciones_de_vida,,,,,,,,
Bueno,21.00,69.74,4.86,59.98,66.61,70.31,72.88,77.40
Malo,21.00,1.06,0.45,0.32,0.84,0.99,1.18,2.44
Muy Bueno,21.00,17.28,6.07,5.68,14.05,16.52,19.07,29.29
Muy Malo,21.00,0.23,0.20,0.08,0.12,0.15,0.27,0.94
Regular,21.00,11.68,4.86,5.35,10.18,11.08,11.75,31.41


Porcentaje y tipo de **condicionesn de vida** mas alto por localidad

In [3]:
df = cv_df.groupby(['localidad'])[['porcentaje']].max()
pd.merge(cv_df, df, how='inner', on=['localidad', 'porcentaje'])

,localidad,condiciones_de_vida,porcentaje
0,Usaquen,Bueno,62.86
1,Chapinero,Bueno,65.18
2,Santa fe,Bueno,67.73
3,San Cristobal,Bueno,77.40
4,Usme,Bueno,71.68
5,Tunjuelito,Bueno,70.29
6,Bosa,Bueno,71.65
7,Kennedy,Bueno,77.00
8,Fontibon,Bueno,71.72
9,Engativa,Bueno,72.88


Localidad con la percepcion de **condicione de vida** *Muy Malo* más alta

In [4]:
pd.DataFrame(cv_df[(cv_df['condiciones_de_vida'] == 'Muy Malo')].max(), columns=['Info'])

,Info
localidad,Usme
condiciones_de_vida,Muy Malo
porcentaje,0.94


## Violencia Intrafamiliar

Valores nulos presentes por cada variable

In [5]:
vi_df.isnull().sum()

year                   0
area                   0
tipo_de_violencia      0
sexo                   0
no_casos               4
poblacion              0
tasa_por_100000      294
dtype: int64

Completar los valores faltantes en la base de datos

In [6]:
# completar valores faltantes en los numeros de casos con 0
vi_df['no_casos'].fillna(0, inplace=True)                

# completar valores faltantes en la tasa por 100mil habitantes con la operacion (no_casos/poblacion)*100000
vi_df['tasa_por_100000'].fillna((vi_df['no_casos'][vi_df['tasa_por_100000'].isna()] / vi_df['poblacion'][vi_df['tasa_por_100000'].isna()])*100000, inplace=True)

In [7]:
vi_df.isnull().sum()

year                 0
area                 0
tipo_de_violencia    0
sexo                 0
no_casos             0
poblacion            0
tasa_por_100000      0
dtype: int64

Cantidad total de casos por **area** y su clasificacion en los percentiles

In [8]:
# se omiten el area de distrito y el tipo de sexo Total general debido que son agregados de los demas grupos

# data frame con la descripción estadisca de la suma de no_casos historicos por localidad 
df = vi_df[(vi_df['area'] != 'Distrito') & (vi_df['sexo'] != 'Total general')].groupby('area')['no_casos'].sum().describe()

# rangos con la clasificación de los percentiles 
rng = pd.cut(vi_df['no_casos'], bins=df.loc['min':'max']).value_counts().index.unique()

# suma de casos historicos por localidad excluyendo los totales
tot = vi_df[(vi_df['area'] != 'Distrito') & (vi_df['sexo'] != 'Total general')].groupby('area')['no_casos'].sum()

ar = []              # lista vacia para almacenar los datos del data-frame

for i, j in zip(tot, tot.index):                                   # extracción de datos para el data-frame
    for y, z in zip(rng, df.loc['25%':'max'].index):
        if i in y:
            ar.append([j, i, z])

p = pd.DataFrame(ar, columns=['localidad', 'casos_totales', 'percentil'])             # definición del data-frame

# insertar columna nivel de gravedad dependiendo de los percentiles en los que se encuentra cada localidad
p['nivel_de_gravedad']= p['percentil'].map({i:j for i,j in zip(df.loc['25%':'max'].index, ['leve', 'regular', 'grave', 'critico'])})

p

,localidad,casos_totales,percentil,nivel_de_gravedad
0,Antonio Nariño,9386.00,25%,leve
1,Barrios Unidos,12720.00,50%,regular
2,Bosa,117487.00,max,critico
3,Chapinero,9698.00,25%,leve
4,Ciudad Bolívar,112209.00,max,critico
5,Engativá,67488.00,75%,grave
6,Fontibón,29600.00,75%,grave
7,Kennedy,96876.00,max,critico
8,Los Mártires,13809.00,50%,regular
9,Puente Aranda,20145.00,50%,regular


Porcentaje de violencia sufrida por **sexo** en todos los años

In [9]:
# total de casos por año
tot_yr = vi_df[(vi_df['area'] != 'Distrito') & (vi_df['sexo'] != 'Total general')].groupby('year')['no_casos'].sum()

# proporción de violencia sufrida por sexo en los distintos años
df = pd.DataFrame(vi_df[(vi_df['area'] != 'Distrito') & (vi_df['sexo'] != 'Total general')].groupby(['year', 'sexo'])['no_casos'].sum()/tot_yr)

df.columns = ['porcentaje_x_yr']         # aignacón del nombre de la columna

df

porcentaje_x_yr
year sexo                    
2012 Hombre              0.36
     Mujer               0.64
2013 Hombre              0.33
     Mujer               0.67
2014 Hombre              0.31
     Mujer               0.69
2015 Hombre              0.30
     Mujer               0.70
2016 Hombre              0.29
     Mujer               0.71
2017 Hombre              0.30
     Mujer               0.70
2018 Hombre              0.29
     Mujer               0.71
2019 Hombre              0.29
     Mujer               0.71
2020 Hombre              0.27
     Mujer               0.73
2021 Hombre              0.27
     Mujer               0.73

## Suicidios


Comparación del numero de casos para cada **sexo** en las edades de 15 a 24 años, en los años _2020_ y _2021_

In [10]:
# funcion de comparación
def fun(yr1, yr2, df):
    
    # lista con los data-frames a comparar
    dfs = [df.pivot_table(index=['year', 'grupo_de_edad'], columns=['sexo'], values='casos', aggfunc='sum').loc[i] for i in [yr1, yr2]]
    
    # union de los data-frames
    r = dfs[0].join(dfs[1], lsuffix='_{}'.format(yr1), rsuffix='_{}'.format(yr2))
    
    return r.loc[r.index.str.contains(r'(([12][1-9]|20) a (1[4-9]|2[0-4]))')]

fun(2020,2021,sc_df)

C:\Users\LUKAS\AppData\Local\Temp\ipykernel_2832\3740283022.py:10: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  return r.loc[r.index.str.contains(r'(([12][1-9]|20) a (1[4-9]|2[0-4]))')]


sexo,Hombre_2020,Mujer_2020,Hombre_2021,Mujer_2021
grupo_de_edad,,,,
15 a 17,14,12,18,8
18 a 19,16,12,16,6
20 a 24,88,24,86,28


In [ ]:
# funcion de comparación
def fun(yr1, yr2, df):
    
    # lista con los data-frames a comparar
    dfs = [df.pivot_table(index=['year', 'grupo_de_edad'], columns=['sexo'], values='casos', aggfunc='sum').loc[i] for i in [yr1, yr2]]
    
    # cambio de nombres en las columnas
    for i,j in zip(dfs, [yr1, yr2]):
        i.columns = ['{}_{}'.format(x, y) for x,y in zip(i.columns, [j]*2)]
        
    # union de los data-frames
    r = pd.concat(dfs, axis=1)
    
    return r.loc[r.index.str.contains(r'(([12][1-9]|20) a (1[4-9]|2[0-4]))')]
    
fun(2021, 2020, sc_df)